In [6]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyarrow
import os
import sys
import json
import urllib.request
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.oauth2 import service_account
import google.auth
import copy
PROJCECT = 'ballosodeuk'


bq = bigquery.Client(project=PROJCECT)
dataset_id = 'airbridge_mart'
table_id = 'web_keyfeatures'
table_ref = f'ballosodeuk.{dataset_id}.{table_id}'

In [ ]:


client = bigquery.Client()
query = """SELECT 
  *
FROM 
  `ballosodeuk.temp.inl_0614-0715`
"""

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

df = query_job.to_dataframe()

In [20]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

# JSON 키 파일 경로 설정
json_keyfile_path = "C:/Users/Owner/Downloads/json/ballosodeuk-4811d21b3452.json"

# 서비스 계정 자격 증명 설정
credentials = service_account.Credentials.from_service_account_file(
    json_keyfile_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform", "https://www.googleapis.com/auth/drive"]
)

# 빅쿼리 클라이언트 생성
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# 쿼리 작성
query = """
SELECT 
  *
FROM 
  `ballosodeuk.temp.inl_0614-0715`
"""

# 쿼리 실행
query_job = client.query(query)

# 결과 불러오기
results = query_job.result()

# 결과를 데이터프레임으로 변환
df = results.to_dataframe()

# 데이터 출력
print(df)


Forbidden: 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: asia-northeast3
Job ID: 6dc87e5b-7c1e-4f66-89f2-39a8021ba734


In [8]:
df = pd.read_csv('/Users/nam-yeong/Downloads/쿠팡 구매데이터의 사본 - 시트1.csv')

In [24]:
product_col = 'productName'

In [25]:
id_1 = pd.read_csv("../../cp_data.csv").iloc[1:4].key[1]
sc_1 = pd.read_csv("../../cp_data.csv").iloc[1:4].value[1]
id_2 = pd.read_csv("../../cp_data.csv").iloc[1:4].key[2]
sc_2 = pd.read_csv("../../cp_data.csv").iloc[1:4].value[2]
id_3 = pd.read_csv("../../cp_data.csv").iloc[1:4].key[3]
sc_3 = pd.read_csv("../../cp_data.csv").iloc[1:4].value[3]

In [34]:
import pandas as pd
import urllib.request
import urllib.parse
import json
import time
import random
from tqdm import tqdm

# Sample DataFrame
# df = pd.read_parquet('path/to/your/parquet/file')
# Assuming df is already defined with 'product_col'

# List of client credentials
client_credentials = [
    {"client_id": id_1, "client_secret": sc_1},
    {"client_id": id_2, "client_secret": sc_2},
    {"client_id": id_3, "client_secret": sc_3},
]

def search_api(client_id, client_secret, query):
    encText = urllib.parse.quote(str(query))  # Ensure the query is a string
    url = "https://openapi.naver.com/v1/search/shop?query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            return json.loads(response_body.decode('utf-8'))
        else:
            print("Error Code:", rescode)
            return rescode
    except urllib.error.HTTPError as e:
        print("HTTPError:", e.code)
        return e.code
    except urllib.error.URLError as e:
        print("URLError:", e.reason)
        return None
    except Exception as e:
        print("General Error:", str(e))
        return None
    
def exponential_backoff(retries):
    delay = (2 ** retries) + random.uniform(0, 1)
    print(f"Exponential backoff: retry {retries}, sleeping for {delay:.2f} seconds")
    time.sleep(delay)

def save_intermediate_results(results, search_terms, start_index):
    result_df = pd.DataFrame.from_dict(results, orient='index', columns=['category1', 'category2', 'category3', 'category4'])
    result_df.reset_index(inplace=True)
    result_df.rename(columns={'index': 'name'}, inplace=True)
    result_df.to_parquet('intermediate_results.parquet')
    
    with open('remaining_search_terms.txt', 'w') as f:
        for term in search_terms[start_index:]:
            f.write(term + "\n")

# Iterate over search terms and perform API search
current_credentials = 0
results = {}
search_terms = list(set(df[product_col].tolist()))  # Ensure the column name matches your DataFrame

for index, term in enumerate(tqdm(search_terms)):
    try:
        retries = 0
        while retries < 5:  # Retry up to 5 times
            client_id = client_credentials[current_credentials]["client_id"]
            client_secret = client_credentials[current_credentials]["client_secret"]
            
            result = search_api(client_id, client_secret, term)
            
            if result == 401:
                print("Unauthorized: Check your API credentials")
                current_credentials = (current_credentials + 1) % len(client_credentials)
                if current_credentials == 0:
                    print("All credentials exhausted. Waiting for 60 seconds before retrying...")
                    time.sleep(60)
                retries += 1
            elif result == 429:
                current_credentials = (current_credentials + 1) % len(client_credentials)
                if current_credentials == 0:
                    print("All credentials exhausted. Waiting for 60 seconds before retrying...")
                    time.sleep(60)
                retries += 1
                exponential_backoff(retries)
            else:
                if isinstance(result, dict):
                    items = result.get('items', [])
                    if len(items) > 0:
                        ct1 = items[0].get('category1', None)
                        ct2 = items[0].get('category2', None)
                        ct3 = items[0].get('category3', None)
                        ct4 = items[0].get('category4', None)
                        results[term] = [ct1, ct2, ct3, ct4]
                    else:
                        results[term] = [None, None, None, None]
                else:
                    results[term] = [None, None, None, None]
                break
            
            time.sleep(1)  # Add a delay between retries
    except Exception as e:
        print(f"An error occurred: {e}")
        save_intermediate_results(results, search_terms, index)
        raise

# Save final results if no error occurs
result_df = pd.DataFrame.from_dict(results, orient='index', columns=['category1', 'category2', 'category3', 'category4'])
result_df.reset_index(inplace=True)
result_df.rename(columns={'index': 'name'}, inplace=True)

result_df.to_parquet('final_results.parquet')



KeyboardInterrupt: 

In [38]:
len(search_terms)

3589

In [41]:
df

,lptag,code,os,adid,timestamp,orderId,wk_uid,click_id,purchase_cancel,productName,productId,gmv,quantity,total_amount
0,AF1177509,balso1sr1,android,bc168aa4-b8bf-4d31-abcc-62739224f1c8,2024-06-15 07:49:33,8.214950e+15,1e932238-13de-413b-a141-b1a3f16cba8b,1e932238-13de-413b-a141-b1a3f16cba8b_null_2024...,purchase,나인테일즈 링클 넥세럼X3+콜라겐 85세럼,7633026753,"70,950",1,"70,950"
1,AF1177509,balso1sr1,android,b420c245-df80-4dfc-8a4f-07d2d400a284,2024-06-15 07:51:32,8.054956e+15,faf3474a-bacc-4d9f-928c-a953b85dd833,faf3474a-bacc-4d9f-928c-a953b85dd833_null_2024...,purchase,에쉬울프 여름 여성용 빅사이즈 화이트 브이넥 반팔 티셔츠,6555162388,"11,200",1,"91,510"
2,AF1177509,balso1sr1,android,b420c245-df80-4dfc-8a4f-07d2d400a284,2024-06-15 07:51:32,8.054956e+15,faf3474a-bacc-4d9f-928c-a953b85dd833,faf3474a-bacc-4d9f-928c-a953b85dd833_null_2024...,purchase,에쉬울프 여름 여성용 빅사이즈 화이트 브이넥 반팔 티셔츠,6555162388,"11,200",1,"91,510"
3,AF1177509,balso1sr1,android,b420c245-df80-4dfc-8a4f-07d2d400a284,2024-06-15 07:51:32,8.054956e+15,faf3474a-bacc-4d9f-928c-a953b85dd833,faf3474a-bacc-4d9f-928c-a953b85dd833_null_2024...,purchase,데일로엔 여성용 미키 브이넥 반팔 티셔츠,1353322288,"19,710",1,"91,510"
4,AF1177509,balso1sr1,android,b420c245-df80-4dfc-8a4f-07d2d400a284,2024-06-15 07:51:32,8.054956e+15,faf3474a-bacc-4d9f-928c-a953b85dd833,faf3474a-bacc-4d9f-928c-a953b85dd833_null_2024...,purchase,"아이리나 [팔찌+발찌세트] 925 실버 작은볼 두줄 팔찌, 발찌 심플 레이어드 팔찌...",8029319980,"15,900",1,"91,510"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4312,AF1177509,balso1sr1,android,fc5a6864-e6e0-494a-b197-f98da130d2f7,2024-07-14 22:33:45,8.924950e+15,NaN,NaN,purchase,[SET] 쿨링 로카 반팔티 검정 1P + 태극기 로카 반바지 검정 1P ROKA ...,7258211158,"22,000",1,"39,300"
4313,AF1177509,balso1sr1,android,fc5a6864-e6e0-494a-b197-f98da130d2f7,2024-07-14 22:33:45,8.924950e+15,NaN,NaN,purchase,나바바 NT1669 여성 세트상품 반팔 반바지 트레이닝 세트,8135067627,"17,300",1,"39,300"
4314,AF1177509,balso1sr1,android,b6d8c902-343a-4939-924a-672f71413d6c,2024-07-14 21:11:53,8.334957e+15,f0f37aea-2fc5-4858-8035-18b11d803916,f0f37aea-2fc5-4858-8035-18b11d803916_null_2024...,purchase,어반티 남녀공용 쿨론 PK 카라 반팔 티셔츠,5976046,"19,000",2,"19,000"
4315,AF6297423,pay010dw2,android,ce1ecbf3-1e4d-428b-8e73-0243f13998c9,2024-07-14 22:25:06,8.054957e+15,NaN,NaN,purchase,농협 GAP 인증 아산 맑은쌀 특등급,7591854478,"28,900",1,"28,900"


In [44]:
result_df.rename(columns={'name':product_col},inplace=True)

In [45]:
merged_df = df.merge(result_df,how='left',on=product_col)
print(len(df))

4317


In [49]:
merged_df.columns

Index(['lptag', 'code', 'os', 'adid', 'timestamp', 'orderId', 'wk_uid',
       'click_id', 'purchase_cancel', 'productName', 'productId', 'gmv',
       'quantity', 'total_amount', 'category1', 'category2', 'category3',
       'category4'],
      dtype='object')

In [51]:
merged_df.to_csv("../result/inl_2024-06-14_2024-07-15_merged.csv",encoding='utf-8-sig')